## Теоретическая часть

#### 1. Ответьте на вопросы:  В чем принципиальное отличие гибридных рекомендательных систем от коллаборативной фильтрации?  Приведите 2-3 примера задач, в которых необходимо использовать гибридные системы.  

Коллаборативная фильтрация вырабатывает рекомендации, основанные на модели предшествующего поведения пользователя. Эта модель может быть построена исключительно на основе поведения данного пользователя и с учетом поведения других пользователей со сходными характеристиками.
Гибридные системы - это как правило комбинирование нескольких алгоритмов в рамках одной платформы. Такой подход позволяет минимизировать проблемы, присущие каждому используемому алгоритму в отдельности. Например, комбинируя коллаборативную фильтрацию и контентный подход мы можем избежать ограничений, свойственных каждой из систем в отдельности. 

Построение рекомендаций фильмов, построение рекомендательной системы большого онлайн магазина, построение рекомендательной системы для сервиса, предоставляющего услуги. Во всеъх этих задачах необходимо использовать гибридные системы.

#### 2.  Прочитайте статью про поиск на hh.ru https://habr.com/ru/company/hh/blog/347276/ Нам интересна именно рекомендательная система, раздел "Производительность системы" можно пропустить Какие основные отличия предложенной системы от тех подходов, которые мы разбирали на семинарах? Какие проблемы могут возникнуть при выводе такой модели в продакшен?


В модели используется ранжирование поисковых запросов. Ранжирование состоит из двух частей: линейной, которая отделяет подходящие вакансии от неподходящих и грубо ранжирует неподходящие и XGBoost, которая используется, чтобы более точно ранжировать подходящие. К полученным при ранжировании признакам добвляют признаки, сравнивающие тексты с учётом текстовых взаимодействий. Все это скормливают модели машинного обучения, которая на выходе дает вероятность отклика на вакансию. 

Система состоит из нескольких компонент и ее будет трудно тюнинговать. Так же достаточно трудоемка отладка такой модели и внасение в нее изменений. 

#### 3. На вебинаре мы рассматривали модель LightFM (https://making.lyst.com/lightfm/docs/lightfm.html). В работе Data Scientist'а важную часть занимает research - исследование существующих архитектур и разбор научных статей, в которых они описываются. Вам предлагается изчуть оригинальную статью про LightFM https://arxiv.org/pdf/1507.08439.pdf и ответить на следующие вопросы:  
* 1) Какой датасет используют авторы?  
* 2) Что используют в качестве признаков?  
* 3) С какими моделями сравнивают LightFM? Опишите их основные идеи кратко  

1) первый датасет это рейтинги фильмов с MovieLens, второй датасет состоит из опубликованных вопросов и ответов на CrossValidated.

2) Для первого датасета фильмы описаны по жанрам и снабжены вектором тегов. Каждая пара тег-фильм - это оценка релевантности (от 0 до 1), обозначающая, насколько точно данный тег описывает фильм. Все рейтинги ниже 4,0 (по шкале от 1 до 5) считаем отрицательными; все рейтинги больше или равные 4.0 положительными. Все теги с рейтингами ниже 0,8 отбрасываются для сохранения только наиболее релевантных тегов.

Для второго датасета набор данных состоит из опубликованных вопросов и ответов. Каждый вопрос сопровождается одним или несколькими из 1032 уникальных тегов. Кроме того, есть метаданные пользователя, которые получают из раздела «Обо мне» в профилях пользователей. Цель рекомендации - задать пользователям вопросы, на которые они могут ответить. 

3)Сравнение производится следующимих моделей: 
* MF модель: обычная модель матричной факторизации со смещением для пользователя и смещением для продукта и сигмоидной функцией.
* LSI-LR: модель, основанная на содержании. Матрица, содержащая количество слов в документе (строки представляют уникальные слова, а столбцы представляют каждый документ), строится из большого фрагмента текста. Далее делается SVD разложение этой матрицы для уменьшения количества строк при сохранении структуры сходства среди столбцов. Из разложения берутся скрытые параметры (темы документов). Каждому пользователю соответствует его вектор в пространстве всех тем. Затем по этим данным строится логистическая регрессия для каждого пользователя.
* LSI-UP: гибридная модель, представляющая профили пользователей, как линейные комбинации векторов представляющих внутренние параметры продуктов. К этим векторам применяют скрытую семантическую индексацию (LSI) (фактически SVD разложение) и получают скрытые параметры (векторы) для пользователей и товаров.
* LightFM (tags): модель LightFM, использующая только теги.
* LightFM (tags + ids): модель LightFM, использующая как теги, так и инентификаторы товаров.
* LightFM (tags + about): модель LightFM с использованием как пользовательских так и продуктовых признаков. Пользовательские признаки доступны только для набора данных CrossValidated. Признаки получаются обработкой раздела пользователей "Обо мне" с помощью представления "мешок слов"(HTML очищается от тегов и небуквенных символов, затем преобразовывается в нижний регистр и
токенизируется на пробелах).

## Практическая часть


In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

import lightfm
from scipy.sparse import csr_matrix

In [4]:
# импорт модулей src.utils и src.recommender

from src.utils import precision_at_k, prefilter_items, postfilter_items, popular_items, gen_dicts
from src.recommender import get_similar_items_recommendation, get_similar_users_recommendation

# читаем данные из файла
data = pd.read_csv('transaction_data.csv')

### 1. Модуль SRC

На вебинаре было рассказано про модуль src. Он приложен в материалах. Скачайте его, изучите структуру, импортируйте функции

### 2. Работа с признаками

У нас есть внешние данные. Что с ними не так? Чего не хватает?  

Проведите исследование внешних данных и составьте какие-нибудь содержательные выводы.  
Формально Вам нужно построить 3+ графиков (scatter plot, hist или что-то иное) и описать, что мы видим (например, товары такой-то категории болле часто покупаются в следующие дни недели или пользователи с большим достатком предпочитают такие-то товары).  
Исследуйте те закономерности, которые Вам интересно, чем менее тривиальный вывод получается, тем лучше! 

### 3. LightFM

У этого алогритма есть множество параметров (item/user_alpha, loss, no_components).  
Проведите эксперименты аналогично дз 3 (подберите гипперпараметры каким удобно способои и постройте графики)  
На выходе необходимо получить pr@5 на валидации (последние 3 недели) > 2%  

У Вас, скорее всего, возникнет проблема со временем обучения. Почему они возникает?    

Попробуйте запустить алгоритм вообще без фичей или используйте только признаки с небольшим числом уникальных категорий. (item_features['commodity_desc'].unique() - 300 уникальных категорий - это очень много)

### *Отбор признаков* * 

Все данные категориальные, при ohe кодировании для товаров признаков становится невероятно много.      
Какие стратегии отбора признаков в классическом ML Вы знаете? Применимы ли они тут?  

Попробйте какие-нибудь стратегии. Удалось ли улучшить качество?

 \* *задание необязательно*
